In [43]:
import pandas as pd
df = pd.read_csv('../data/data18.csv')

In [2]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

In [3]:
y = df['label'] # We specify y now because we will drop this column later
df.drop('timestamp', inplace=True, axis=1) # dropping timestamp column
df.drop('ip_src', inplace=True, axis=1) # dropping ip_src column
df.drop('ip_dst', inplace=True, axis=1) # dropping ip_dst column
df.drop('label', inplace=True, axis=1) # dropping label column
df.drop('attack_type', inplace=True, axis=1) # dropping attack_type column

In [4]:
# Min-Max Function
def min_max_normalize(column):
    min_val = column.min()
    max_val = column.max()
    column_normalized = (column - min_val) / (max_val - min_val)
    return column_normalized

In [5]:
x = pd.DataFrame()
for i in df.columns:
  x = pd.concat([x, min_max_normalize(df[i])], axis=1)
x.astype('float16')
x = x.fillna(0)

In [6]:
x

,datapath_id,flow_id,tp_src,tp_dst,ip_proto,icmp_code,icmp_type,flow_duration_sec,flow_duration_nsec,idle_timeout,hard_timeout,flags,packet_count,byte_count,packet_count_per_second,packet_count_per_nsecond,byte_count_per_second,byte_count_per_nsecond
0,0.75,0.000004,0.000000,0.000000,0.0000,0.25,0.111111,0.06,0.447447,0.0,0.0,0.0,7.475427e-07,1.043540e-08,6.906390e-07,3.409512e-09,1.004996e-08,2.425123e-11
1,0.50,0.046155,0.077058,0.566128,0.3125,0.00,0.000000,0.06,0.439439,0.0,0.0,0.0,1.368855e-02,1.286934e-04,1.264657e-02,6.357067e-05,1.239401e-04,3.045259e-07
2,0.50,0.000297,0.547097,0.079739,0.3125,0.00,0.000000,0.06,0.472472,0.0,0.0,0.0,5.973778e-02,4.850407e-01,5.519048e-02,2.580304e-04,4.671256e-01,1.067501e-03
3,0.50,0.006225,0.547097,0.079739,0.0000,0.25,1.000000,0.06,0.474474,0.0,0.0,0.0,8.970513e-07,1.201136e-08,8.287668e-07,3.858359e-09,1.156771e-08,2.632364e-11
4,0.50,0.004644,0.547097,0.079739,0.0000,0.25,0.111111,0.06,0.443443,0.0,0.0,0.0,7.475427e-07,1.043540e-08,6.906390e-07,3.440298e-09,1.004996e-08,2.447020e-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1382124,0.25,0.000037,0.078218,0.000000,1.0000,0.00,0.000000,0.14,0.922923,0.0,0.0,0.0,1.943611e-06,4.485091e-08,7.695692e-07,4.297760e-09,1.851186e-08,5.053272e-11
1382125,0.25,0.000037,0.080583,0.000000,1.0000,0.00,0.000000,0.14,0.837838,0.0,0.0,0.0,7.475427e-07,1.725035e-08,2.959882e-07,1.820851e-09,7.119945e-09,2.140941e-11
1382126,0.25,0.000037,0.336034,0.000000,1.0000,0.00,0.000000,0.03,0.782783,0.0,0.0,0.0,5.980342e-07,1.380028e-08,1.105022e-06,1.559132e-09,2.658113e-08,1.833215e-11
1382127,0.25,0.000037,0.076707,0.000000,1.0000,0.00,0.000000,0.15,0.055055,0.0,0.0,0.0,2.541645e-06,5.865119e-08,9.392691e-07,9.421412e-08,2.259396e-08,1.107762e-09


In [8]:
# Example usage:
# Assuming X is the feature matrix and y is the target vector
X = x

n_features_to_select = 16

# Create a Random Forest Classifier using gini index
clf = RandomForestClassifier(criterion='gini')

# Perform Recursive Feature Elimination
rfe = RFE(estimator=clf, n_features_to_select=n_features_to_select)
rfe.fit(X, y)

# Get the selected features
selected_features = rfe.support_

print(selected_features)

[ True  True  True  True  True  True  True  True  True  True False False
  True  True  True  True  True  True]


In [9]:
features = []
for i, feature in enumerate(selected_features):
  if feature:
    features.append(X.columns[i])

In [10]:
print("Selected Features are:")
features

Selected Features are:


['datapath_id',
 'flow_id',
 'tp_src',
 'tp_dst',
 'ip_proto',
 'icmp_code',
 'icmp_type',
 'flow_duration_sec',
 'flow_duration_nsec',
 'idle_timeout',
 'packet_count',
 'byte_count',
 'packet_count_per_second',
 'packet_count_per_nsecond',
 'byte_count_per_second',
 'byte_count_per_nsecond']